# [Optional] Introduction to Strands Agents

This optional module introduces the Strands Agents SDK for building AI agents. Complete this if you're new to agentic AI development before proceeding to the main Developer Journey labs.

## Learning Objectives

By the end of this notebook, you will be able to:
- Create a basic agent using the Strands Agents SDK
- Add built-in and custom tools to extend agent capabilities
- Understand the agent execution loop and metrics
- Deploy an agent to Amazon Bedrock AgentCore Runtime

**Duration**: ~30 minutes

## What is Strands Agents?

[Strands Agents](https://strandsagents.com/) is an open-source SDK for building AI agents with a model-driven approach. Instead of manually orchestrating complex workflows, Strands lets the model decide when and how to use tools.

### Core Components

| Component | Description |
|-----------|-------------|
| **Model** | The LLM that powers reasoning (e.g., Claude on Bedrock) |
| **System Prompt** | Instructions defining agent behavior and personality |
| **Tools** | Functions the agent can call to take actions |

### Agent Loop Architecture

```
┌──────────────┐     ┌─────────────────────────────────┐     ┌──────────────┐
│  User Input  │ ──> │         Strands Agent           │ ──> │   Response   │
└──────────────┘     │                                 │     └──────────────┘
                     │  ┌─────────────────────────┐    │
                     │  │      Agent Loop         │    │
                     │  │  1. Reason (LLM)        │    │
                     │  │  2. Select Tool         │    │
                     │  │  3. Execute Tool        │    │
                     │  │  4. Repeat or Respond   │    │
                     │  └─────────────────────────┘    │
                     └─────────────────────────────────┘
```

The agent autonomously decides whether to respond directly or use tools based on the user's request.

---

## 1. Setup

First, install the Strands Agents SDK and configure AWS credentials.

In [ ]:
# Install Strands Agents SDK
!pip install strands-agents strands-agents-tools --quiet

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configuration
REGION = os.getenv('AWS_DEFAULT_REGION', 'us-east-1')
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"

print(f"Region: {REGION}")
print(f"Model: {MODEL_ID}")

---

## 2. Your First Agent

Create a simple agent with just a few lines of code. The agent uses Claude on Amazon Bedrock for reasoning.

In [ ]:
from strands import Agent

# Create a basic agent
agent = Agent(
    model=MODEL_ID,
    system_prompt="You are a helpful assistant. Be concise and accurate."
)

# Invoke the agent
_ = agent("What is Amazon Bedrock in one sentence?")

<div class="alert alert-block alert-info">
<b>Note:</b> The agent automatically handles the conversation with the model. You simply call the agent like a function with your input.
</div>

---

## 3. Adding Tools

Tools extend agent capabilities beyond text generation. The agent autonomously decides when to use tools based on the user's request.

### Built-in Tools

Strands provides common tools out of the box via the `strands-agents-tools` package:

In [ ]:
from strands import Agent
from strands_tools import calculator, current_time

# Create agent with built-in tools
agent_with_tools = Agent(
    model=MODEL_ID,
    tools=[calculator, current_time],
    system_prompt="You are a helpful assistant with access to a calculator and clock."
)

# The agent automatically uses the calculator tool
_ = agent_with_tools("What is 15% of 250?")

In [ ]:
# The agent uses the current_time tool
_ = agent_with_tools("What time is it now?")

### Custom Tools

Create domain-specific tools using the `@tool` decorator. This is essential for building agents that interact with your business systems.

In [ ]:
from strands import Agent, tool

@tool
def lookup_order(order_id: str) -> str:
    """Look up order status by order ID.
    
    Args:
        order_id: The order ID to look up (e.g., ORD-12345)
    
    Returns:
        Order status information
    """
    # Simulated order database
    orders = {
        "ORD-12345": {"status": "Shipped", "eta": "2 days", "carrier": "FedEx"},
        "ORD-67890": {"status": "Processing", "eta": "5 days", "carrier": "Pending"},
    }
    
    if order_id in orders:
        order = orders[order_id]
        return f"Order {order_id}: {order['status']}, ETA: {order['eta']}, Carrier: {order['carrier']}"
    return f"Order {order_id} not found"


@tool
def check_inventory(product_sku: str) -> str:
    """Check inventory for a product SKU.
    
    Args:
        product_sku: The product SKU to check
    
    Returns:
        Inventory status
    """
    # Simulated inventory
    inventory = {
        "SKU-KB-001": {"name": "Wireless Keyboard", "qty": 150, "warehouse": "Seattle"},
        "SKU-MS-002": {"name": "Gaming Mouse", "qty": 0, "warehouse": "N/A"},
    }
    
    if product_sku in inventory:
        item = inventory[product_sku]
        status = "In Stock" if item['qty'] > 0 else "Out of Stock"
        return f"{item['name']}: {status} ({item['qty']} units in {item['warehouse']})"
    return f"Product {product_sku} not found"


print("Custom tools defined: lookup_order, check_inventory")

<div class="alert alert-block alert-warning">
<b>Key Insight:</b> The docstring in your tool function is critical - it tells the model what the tool does and when to use it. Write clear, descriptive docstrings for better tool selection.
</div>

---

## 4. Customer Support Agent Example

Let's build a customer support agent similar to what we'll use in the main Developer Journey labs.

In [ ]:
support_agent = Agent(
    model=MODEL_ID,
    tools=[lookup_order, check_inventory],
    system_prompt="""You are a customer support agent for CloudCommerce.

Your responsibilities:
- Help customers check order status using the lookup_order tool
- Check product inventory using the check_inventory tool
- Be friendly, professional, and helpful

Always use the available tools to get accurate information.
If you cannot find information, apologize and offer alternatives."""
)

print("Customer support agent created with 2 tools")

In [ ]:
# Test order lookup
_ = support_agent("Where is my order ORD-12345?")

In [ ]:
# Test inventory check
_ = support_agent("Do you have the wireless keyboard SKU-KB-001 in stock?")

In [ ]:
# Test handling unknown order
_ = support_agent("What's the status of order ORD-99999?")

---

## 5. Understanding Agent Execution

After running an agent, you can inspect execution metrics to understand what happened.

In [ ]:
result = support_agent("Check if SKU-MS-002 is available")

# Access metrics from the result
print("\n" + "=" * 50)
print("EXECUTION METRICS")
print("=" * 50)

if hasattr(result, 'metrics'):
    metrics = result.metrics
    if hasattr(metrics, 'accumulated_usage'):
        usage = metrics.accumulated_usage
        print(f"Input tokens:  {usage.get('inputTokens', 'N/A')}")
        print(f"Output tokens: {usage.get('outputTokens', 'N/A')}")
        print(f"Total tokens:  {usage.get('totalTokens', 'N/A')}")
else:
    print("Metrics not available in this response format")

---

## 6. Deploy to Amazon Bedrock AgentCore Runtime

Amazon Bedrock AgentCore Runtime provides a managed, serverless environment for running agents in production. In this section, we'll deploy our customer support agent to AgentCore Runtime.

<div class="alert alert-block alert-warning">
<b>Important:</b> The AgentCore Starter Toolkit (<code>bedrock-agentcore-starter-toolkit</code>) used in this section is designed for <b>development and learning purposes only</b>. For production deployments, use proper Infrastructure-as-Code (IaC) tools like AWS CDK, CloudFormation, or Terraform to provision AgentCore resources with appropriate security, monitoring, and governance controls.
</div>

### What is AgentCore Runtime?

AgentCore Runtime is a fully managed service that hosts your AI agents. When you deploy an agent, AgentCore:

1. **Packages your code** into a container or deployment bundle
2. **Creates AWS resources** (IAM roles, S3 buckets, etc.)
3. **Hosts your agent** in a serverless environment
4. **Provides an API endpoint** for invoking your agent

### Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                        Amazon Bedrock AgentCore                             │
│  ┌─────────────────────────────────────────────────────────────────────┐   │
│  │                      AgentCore Runtime                               │   │
│  │                                                                      │   │
│  │   ┌──────────────┐    ┌──────────────┐    ┌──────────────┐          │   │
│  │   │   Agent      │    │   Agent      │    │   Agent      │          │   │
│  │   │  Instance 1  │    │  Instance 2  │    │  Instance N  │   ...    │   │
│  │   │  (Session A) │    │  (Session B) │    │  (Session N) │          │   │
│  │   └──────────────┘    └──────────────┘    └──────────────┘          │   │
│  │          │                   │                   │                   │   │
│  │          └───────────────────┼───────────────────┘                   │   │
│  │                              ▼                                       │   │
│  │                    ┌──────────────────┐                              │   │
│  │                    │  Amazon Bedrock  │                              │   │
│  │                    │   Foundation     │                              │   │
│  │                    │     Models       │                              │   │
│  │                    └──────────────────┘                              │   │
│  └─────────────────────────────────────────────────────────────────────┘   │
│                                                                             │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐        │
│  │   Memory    │  │    Code     │  │Observability│  │   Identity  │        │
│  │  (Optional) │  │ Interpreter │  │  (Tracing)  │  │   Gateway   │        │
│  └─────────────┘  └─────────────┘  └─────────────┘  └─────────────┘        │
└─────────────────────────────────────────────────────────────────────────────┘
         ▲                                                    │
         │              InvokeAgentRuntime API                │
         │                                                    ▼
┌─────────────────┐                                  ┌─────────────────┐
│   Your Client   │ ◄────────────────────────────────│   API Gateway   │
│   Application   │         Response                 │   / Lambda      │
└─────────────────┘                                  └─────────────────┘
```

### Key Benefits

| Feature | Description |
|---------|-------------|
| **Serverless** | No infrastructure to manage - pay only for what you use |
| **Auto-scaling** | Handles variable load automatically |
| **Session Isolation** | Each conversation runs in isolated context |
| **IAM Integration** | Native AWS security and access control |
| **Framework Agnostic** | Works with Strands, LangGraph, CrewAI, and more |

### Step 1: Install AgentCore Starter Toolkit

In [ ]:
# Install the AgentCore SDK and Starter Toolkit
!pip install bedrock-agentcore bedrock-agentcore-starter-toolkit --quiet

### Step 2: Create Agent File

The AgentCore Runtime requires your agent to be wrapped with `BedrockAgentCoreApp`. This provides the HTTP interface that AgentCore uses to invoke your agent.

In [ ]:
%%writefile demo_agent.py
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

@tool
def lookup_order(order_id: str) -> str:
    """Look up order status by order ID.
    
    Args:
        order_id: The order ID to look up (e.g., ORD-12345)
    
    Returns:
        Order status information
    """
    orders = {
        "ORD-12345": {"status": "Shipped", "eta": "2 days", "carrier": "FedEx"},
        "ORD-67890": {"status": "Processing", "eta": "5 days", "carrier": "Pending"},
    }
    if order_id in orders:
        order = orders[order_id]
        return f"Order {order_id}: {order['status']}, ETA: {order['eta']}, Carrier: {order['carrier']}"
    return f"Order {order_id} not found"

agent = Agent(
    model="us.anthropic.claude-sonnet-4-20250514-v1:0",
    tools=[lookup_order],
    system_prompt="You are a helpful customer support agent. Use the lookup_order tool to check order status."
)

@app.entrypoint
def invoke(payload):
    """Main entrypoint for AgentCore Runtime."""
    user_input = payload.get("prompt", "")
    response = agent(user_input)
    # Extract text from response
    if hasattr(response, 'message') and response.message:
        content = response.message.get('content', [])
        if content and isinstance(content, list):
            return content[0].get('text', str(response))
    return str(response)

if __name__ == "__main__":
    app.run()

In [ ]:
%%writefile demo_requirements.txt
strands-agents
bedrock-agentcore

### Step 3: Test Locally (Optional)

Before deploying, you can test the agent locally. This requires two terminals:

**Terminal 1** - Start the agent server:
```bash
python demo_agent.py
```

**Terminal 2** - Test with curl:
```bash
curl -X POST http://localhost:8080/invocations -H "Content-Type: application/json" -d '{"prompt": "Where is my order ORD-12345?"}'
```

<div class="alert alert-block alert-info">
<b>Note:</b> Local testing is optional. You can skip directly to deployment if preferred.
</div>

### Step 4: Configure the Agent

Use the AgentCore CLI to configure your agent. This creates a `.bedrock_agentcore.yaml` configuration file.

In [ ]:
# Configure the agent for deployment
# --disable-memory skips memory configuration for this simple demo
# --non-interactive avoids prompts and Unicode display issues on Windows
!set PYTHONIOENCODING=utf-8 && agentcore configure -e demo_agent.py -rf demo_requirements.txt --disable-memory --non-interactive

### Step 5: Deploy to AgentCore Runtime

Deploy your agent to AWS. This will:
1. Create an IAM execution role
2. Package your code and upload to S3
3. Create the AgentCore Runtime
4. Return the Agent ARN for invocation

<div class="alert alert-block alert-warning">
<b>Note:</b> Deployment takes 2-5 minutes. Save the Agent ARN from the output - you'll need it to invoke the agent.
</div>

In [ ]:
# Deploy to AgentCore Runtime
!set PYTHONIOENCODING=utf-8 && agentcore launch

### Step 6: Test the Deployed Agent

Test your deployed agent using the AgentCore CLI:

In [ ]:
# Test the deployed agent
!set PYTHONIOENCODING=utf-8 && agentcore invoke "{\"prompt\": \"Where is my order ORD-12345?\"}"

You can also invoke programmatically using boto3:

In [ ]:
# Programmatic invocation example (update AGENT_ARN from launch output)
import boto3
import json

# Replace with your agent ARN from agentcore launch output
AGENT_ARN = "<YOUR_AGENT_ARN>"  # e.g., arn:aws:bedrock:us-west-2:123456789012:agent-runtime/abc123

if AGENT_ARN != "<YOUR_AGENT_ARN>":
    client = boto3.client("bedrock-agentcore", region_name="us-west-2")
    payload = json.dumps({"prompt": "Where is my order ORD-12345?"}).encode()
    
    response = client.invoke_agent_runtime(
        agentRuntimeArn=AGENT_ARN,
        payload=payload
    )
    
    # Handle streaming response
    if "text/event-stream" in response.get("contentType", ""):
        for line in response["response"].iter_lines():
            if line:
                print(line.decode("utf-8"))
    else:
        print(response)
else:
    print("Update AGENT_ARN with the ARN from agentcore launch output")

### Step 7: Clean Up

When you're done testing, delete the deployed resources to avoid charges:

In [ ]:
# Delete the AgentCore Runtime and associated resources
!set PYTHONIOENCODING=utf-8 && agentcore destroy

In [ ]:
# Clean up local files created for this demo
import os

for f in ['demo_agent.py', 'demo_requirements.txt']:
    if os.path.exists(f):
        os.remove(f)
        print(f"Removed {f}")

# Remove .bedrock_agentcore.yaml if it exists
if os.path.exists('.bedrock_agentcore.yaml'):
    os.remove('.bedrock_agentcore.yaml')
    print("Removed .bedrock_agentcore.yaml")

print("\nLocal cleanup complete!")

---

## Summary

| Concept | Description |
|---------|-------------|
| **Agent** | Model + System Prompt + Tools |
| **Tools** | Functions the agent can call |
| **@tool decorator** | Creates custom tools from Python functions |
| **Agent Loop** | Reason → Select Tool → Execute → Repeat/Respond |
| **AgentCore Runtime** | Managed serverless environment for production |

### Key Takeaways

1. **Simple API**: Create agents with just 3 components (model, prompt, tools)
2. **Tool-driven**: The model decides when and how to use tools
3. **Custom tools**: Use `@tool` decorator for domain-specific functionality
4. **Production-ready**: AgentCore Runtime handles scaling and infrastructure

---

**Next**: Continue to the main Developer Journey labs to build a complete customer support system with observability and evaluation.